In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
VALIDATION_SIZE = 1000
df = pd.read_csv("train.csv", nrows=10000)


In [4]:
df = df.fillna(0)

In [5]:
df

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,0.000000,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,0.000000,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,0.000000,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,0.000000,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,0.000000,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0.420165,0.001847,0.016171,0.039813,0.061120,0.051909,-1,-0.803361,-1.687988,...,-2.735500,-0.603269,-1.588024,-0.614948,-2.783072,-0.882524,-1.890361,-0.789504,-1.662447,9995
9996,1,2.852259,-0.000712,-0.004864,-0.009795,-0.018601,-0.019537,1,-0.702446,-1.161989,...,-0.861067,-0.883083,-0.165111,-0.779975,-0.422061,-0.968161,0.167733,-0.920668,-0.010404,9996
9997,1,1.389255,0.002383,0.004190,0.002356,0.006833,0.009608,1,0.130875,-0.608811,...,0.276647,-0.001041,0.269809,-0.036464,0.415885,0.056424,0.422491,-0.036431,0.176114,9997
9998,1,0.085839,-0.003532,-0.037491,-0.053815,-0.052844,-0.065972,1,-3.172026,-3.093182,...,0.027756,0.940655,-0.102939,0.714865,-0.302707,1.022478,-0.315486,0.806448,-0.441524,9998


In [6]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [7]:
df.iloc[2]["resp"]

0.0239701262920541

In [8]:
df.iloc[2]["feature_0":"feature_129"].values

array([-1.        ,  0.81278043, -0.25615584,  0.80646267,  0.4002206 ,
       -0.61418779, -0.35480005,  0.        ,  0.        ,  5.44826084,
        2.66802875,  0.        ,  0.        ,  3.83634173,  2.18325752,
        3.90269776,  3.04543074,  0.        ,  0.        , -1.1410816 ,
       -0.97996232,  0.        ,  0.        , -1.15758467, -0.96680317,
       -1.43097261, -1.10343163,  0.        ,  0.        ,  5.13155872,
        4.31471422,  0.        ,  0.        ,  4.2263413 ,  3.17364035,
        5.99151344,  4.14229764, -0.16792715, -0.12477773,  0.74932579,
        0.71582445, -0.03900658,  0.18632062,  2.32388738,  0.16226144,
        0.23798736, -0.35022138, -0.13803296, -0.51628111, -0.7035434 ,
       -0.55695406, -0.81690971, -0.45584133,  2.3832265 ,  3.47441639,
        0.        ,  0.88324663, -0.08424736,  0.62256055,  0.95361872,
       -1.12877427, -0.94650748, -1.47076236, -1.59458685, -4.34619924,
       -2.27667794, -1.41765152, -2.16636207, -2.07710993,  0.69

In [9]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


/home/vaclavmatejka/adata/anaconda3/envs/coursera/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [10]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [11]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [12]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [13]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

In [14]:
model.eval()

Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [15]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

(tensor(-0.0036), tensor([0.0013], device='cuda:0', grad_fn=<SelectBackward>))

In [16]:
accuracy_list

[tensor(0.0186, device='cuda:0'),
 tensor(0.0153, device='cuda:0'),
 tensor(0.0061, device='cuda:0'),
 tensor(0.0036, device='cuda:0'),
 tensor(0.0037, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0044, device='cuda:0'),
 tensor(0.0044, device='cuda:0'),
 tensor(0.0035, device='cuda:0'),
 tensor(0.0068, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0051, device='cuda:0'),
 tensor(0.0029, device='cuda:0'),
 tensor(0.0022, device='cuda:0'),
 tensor(0.0020, device='cuda:0'),
 tensor(0.0028, device='cuda:0'),
 tensor(0.0023, device='cuda:0'),
 tensor(0.0026, device='cuda:0'),
 tensor(0.0024, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0022, device='cuda:0'),
 tensor(0.0030, device='cuda:0'),
 tensor(0.0020, device='cuda:0'),
 tensor(0.0017, device='cuda:0'),
 tensor(0.0014, device='cuda:0'),
 tensor(0.0021, device='cuda:0'),
 tensor(0.0018, device='cuda:0'),
 tensor(0.0017, device='cuda:0'),
 tensor(0.0019, device='cuda:0'),
 tensor(0.0028

In [17]:
loss_list

[tensor(1.7777, device='cuda:0'),
 tensor(260.3978, device='cuda:0'),
 tensor(24.2435, device='cuda:0'),
 tensor(138.1919, device='cuda:0'),
 tensor(91.5306, device='cuda:0'),
 tensor(26.3294, device='cuda:0'),
 tensor(12.3440, device='cuda:0'),
 tensor(25.0281, device='cuda:0'),
 tensor(25.7195, device='cuda:0'),
 tensor(17.7737, device='cuda:0'),
 tensor(10.2398, device='cuda:0'),
 tensor(8.9544, device='cuda:0'),
 tensor(7.7700, device='cuda:0'),
 tensor(7.5105, device='cuda:0'),
 tensor(5.6331, device='cuda:0'),
 tensor(3.2646, device='cuda:0'),
 tensor(2.7660, device='cuda:0'),
 tensor(2.6691, device='cuda:0'),
 tensor(3.4571, device='cuda:0'),
 tensor(4.3666, device='cuda:0'),
 tensor(4.8957, device='cuda:0'),
 tensor(4.0818, device='cuda:0'),
 tensor(2.4752, device='cuda:0'),
 tensor(0.9717, device='cuda:0'),
 tensor(0.5463, device='cuda:0'),
 tensor(0.7192, device='cuda:0'),
 tensor(1.0505, device='cuda:0'),
 tensor(1.6187, device='cuda:0'),
 tensor(1.4869, device='cuda:0'),
 t